In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold

import os
import sys
sys.path.append('..')

from utils import calc_wap, calc_wap2, log_return, realized_volatility, count_unique, calc_mean_importance, calc_model_importance, plot_importance, rmspe, feval_RMSPE
from preprocess import preprocessor, get_time_stock

In [24]:
NB = "005"
feature_path = "../output/feature/nb" + NB
model_path = "../output/model/nb" + NB
if not os.path.isdir(feature_path):
    os.makedirs(feature_path)
if not os.path.isdir(model_path):
    os.makedirs(model_path)


In [3]:
data_dir = '../input/optiver-realized-volatility-prediction/'

# トレーニングデータ

In [4]:
%%time
train = pd.read_csv(data_dir + 'train.csv')
train_ids = train.stock_id.unique()
df_train = preprocessor(list_stock_ids=train_ids, is_train=True)
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id', 'target']]
df_train = train.merge(df_train, on=['row_id'], how='left')
df_train.head()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  4.9min finished


CPU times: user 6.05 s, sys: 2.04 s, total: 8.09 s
Wall time: 4min 59s


,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0-5,0.004136,0.004499,7.613599e-06,0.000260,0.002292,0.006999,0.000008,0.000404,0.002325,0.000388,0.000295,0.117051,0.000852,0.000211,0.257255,0.000176,0.000162,0.053006,-0.000151,0.000126,-0.045557,134.894040,107.260583,40738,323.496689,138.101214,97696,1.003725,0.000693,303.125061,0.001721,-0.000005,0.000210,-0.000361,0.004114,0.000001,0.000503,0.000068,0.000366,0.000277,0.024868,0.000783,0.000181,0.053236,0.000262,0.000178,0.017790,-0.000166,0.000126,-0.011274,141.470588,84.467864,9620,263.941176,116.940077,17948,1.003482,0.000514,68.236749,0.002953,1.131529e-06,0.000251,0.000157,0.004863,0.000002,0.000414,0.000274,0.000372,0.000273,0.051757,0.000822,0.000237,0.114272,0.000223,0.000173,0.030976,-0.000162,0.000131,-0.022548,137.158273,97.898813,19065,294.928058,136.527199,40995,1.003753,0.000487,139.521722,0.003796,0.000001,0.000250,0.000276,0.006087,1.295471e-08,0.000400,0.000003,0.000397,0.000281,0.091997,0.000858,0.000221,0.199058,0.000188,0.000165,0.043697,-0.000147,0.000120,-0.034024,123.586207,103.533216,28672,327.431034,142.761068,75964,1.003832,0.000445,232.888919,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0

# テストデータ

In [5]:
%%time
test = pd.read_csv(data_dir + 'test.csv')
test_ids = test.stock_id.unique()
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


CPU times: user 43.4 ms, sys: 7.75 ms, total: 51.2 ms
Wall time: 272 ms


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished


# ターゲットエンコーディング

In [6]:
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean()
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean)

tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits=10, shuffle=True, random_state=55)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
    
    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

# 特徴量追加　time_id、stock_idで集約して統計量とったもの(realized_volatilityに関して)

In [9]:
df_train['time_id'] = df_train['row_id'].apply(lambda x:x.split('-')[1])
df_train['time_id']
df_train = get_time_stock(df_train)
df_test = get_time_stock(df_test)
df_train = df_train.drop(['time_id'], axis=1)

In [10]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id,stock_id_target_enc,log_return_realized_volatility_mean_stock_x,log_return_realized_volatility_std_stock_x,log_return_realized_volatility_max_stock_x,log_return_realized_volatility_min_stock_x,log_return2_realized_volatility_mean_stock_x,log_return2_realized_volatility_std_stock_x,log_return2_realized_volatility_max_stock_x,log_return2_realized_volatility_min_stock_x,log_return_realized_volatility_450_mean_stock_x,log_return_realized_volatility_450_std_stock_x,log_return_realized_volatility_450_max_stock_x,log_return_realized_volatility_450_min_stock_x,log_return2_realized_volatility_450_mean_stock_x,log_return2_realized_volatility_450_std_stock_x,...,log_return_realized_volatility_450_max_time_max_time,log_return_realized_volatility_450_max_time_min_time,log_return_realized_volatility_450_min_time_mean_time,log_return_realized_volatility_450_min_time_std_time,log_return_realized_volatility_450_min_time_max_time,log_return_realized_volatility_450_min_time_min_time,log_return2_realized_volatility_450_mean_time_mean_time,log_return2_realized_volatility_450_mean_time_std_time,log_return2_realized_volatility_450_mean_time_max_time,log_return2_realize

In [11]:
df_test.head()

,stock_id,time_id,row_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id_target_enc,log_return_realized_volatility_mean_stock_x,log_return_realized_volatility_std_stock_x,log_return_realized_volatility_max_stock_x,log_return_realized_volatility_min_stock_x,log_return2_realized_volatility_mean_stock_x,log_return2_realized_volatility_std_stock_x,log_return2_realized_volatility_max_stock_x,log_return2_realized_volatility_min_stock_x,log_return_realized_volatility_450_mean_stock_x,log_return_realized_volatility_450_std_stock_x,log_return_realized_volatility_450_max_stock_x,log_return_realized_volatility_450_min_stock_x,log_return2_realized_volatility_450_mean_stock_x,log_return2_realized_volatility_450_std_stock_x,...,log_return_realized_volatility_450_max_time_max_time,log_return_realized_volatility_450_max_time_min_time,log_return_realized_volatility_450_min_time_mean_time,log_return_realized_volatility_450_min_time_std_time,log_return_realized_volatility_450_min_time_max_time,log_return_realized_volatility_450_min_time_min_time,log_return2_realized_volatility_450_mean_time_mean_time,log_return2_realized_volatility_450_mean_time_std_time,log_return2_realized_volatility_450_mean_time_max_time,log_return2_realiz

# LightGBM

In [12]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [13]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [14]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8, # 利用する特徴量の割合
      'bagging_fraction': 0.8,
  }

In [15]:
kf = KFold(n_splits=5, random_state=55, shuffle=True)
oof = pd.DataFrame()
models = []
scores = 0.0

In [16]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'Fold: {fold+1}')
    X_train, y_train = X.loc[trn_idx], y.loc[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y.loc[val_idx]
    
    # RMSPEで最適化を行うため? RMSEの損失関数に1/yi^2の重み付けをすればOK
    # https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/250324
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight=weights)
    
    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference=lgbm_train, weight=weights)
    
    model = lgbm.train(params=params,
                  train_set=lgbm_train,
                  valid_sets=[lgbm_train, lgbm_valid],
                  num_boost_round=5000,         
                  feval=feval_RMSPE,
                  verbose_eval=100,
                  categorical_feature = ['stock_id']                
                 )
    
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)

Fold: 1


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.969136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 122999
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 826


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001800
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000654324	training's RMSPE: 0.30286	valid_1's rmse: 0.00065878	valid_1's RMSPE: 0.30479
[200]	training's rmse: 0.000520219	training's RMSPE: 0.24078	valid_1's rmse: 0.000528811	valid_1's RMSPE: 0.24466
[300]	training's rmse: 0.000487111	training's RMSPE: 0.22546	valid_1's rmse: 0.000497357	valid_1's RMSPE: 0.2301
[400]	training's rmse: 0.000474401	training's RMSPE: 0.21958	valid_1's rmse: 0.000484724	valid_1's RMSPE: 0.22426
[500]	training's rmse: 0.000466685	training's RMSPE: 0.21601	valid_1's rmse: 0.000477324	valid_1's RMSPE: 0.22084
[600]	training's rmse: 0.000460486	training's RMSPE: 0.21314	valid_1's rmse: 0.000471612	valid_1's RMSPE: 0.21819
[700]	training's rmse: 0.000455235	training's RMSPE: 0.21071	valid_1's rmse: 0.000467064	valid_1's RMSPE: 0.21609
[800]	training's rmse: 0.000450632	training's RMSPE: 0.20858	valid_1's rmse: 0.00046326	va

...

[LightGBM] [Info] Start training from score 0.001798
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000653555	training's RMSPE: 0.30285	valid_1's rmse: 0.000661081	valid_1's RMSPE: 0.30441
[200]	training's rmse: 0.000519544	training's RMSPE: 0.24075	valid_1's rmse: 0.000529132	valid_1's RMSPE: 0.24365
[300]	training's rmse: 0.000486567	training's RMSPE: 0.22547	valid_1's rmse: 0.000496862	valid_1's RMSPE: 0.22879
[400]	training's rmse: 0.000473866	training's RMSPE: 0.21959	valid_1's rmse: 0.000485186	valid_1's RMSPE: 0.22342
[500]	training's rmse: 0.000465746	training's RMSPE: 0.21582	valid_1's rmse: 0.000477846	valid_1's RMSPE: 0.22004
[600]	training's rmse: 0.000459547	training's RMSPE: 0.21295	valid_1's rmse: 0.000472778	valid_1's RMSPE: 0.2177
[700]	training's rmse: 0.000454291	training's RMSPE: 0.21052	valid_1's rmse: 0.000468748	valid_1's RMSPE: 0.21585
[800]	training's rmse: 0.000449818	training's RMSPE: 0.20844	valid_1's rmse: 0.000465503	

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.078998 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 122999
[LightGBM] [Info] Number of data points in the train set: 343146, number of used features: 826


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001801
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000655111	training's RMSPE: 0.30305	valid_1's rmse: 0.000652941	valid_1's RMSPE: 0.30275
[200]	training's rmse: 0.000520774	training's RMSPE: 0.24091	valid_1's rmse: 0.000521875	valid_1's RMSPE: 0.24198
[300]	training's rmse: 0.000487713	training's RMSPE: 0.22561	valid_1's rmse: 0.000490446	valid_1's RMSPE: 0.22741
[400]	training's rmse: 0.000475043	training's RMSPE: 0.21975	valid_1's rmse: 0.000478762	valid_1's RMSPE: 0.22199
[500]	training's rmse: 0.00046748	training's RMSPE: 0.21625	valid_1's rmse: 0.000472338	valid_1's RMSPE: 0.21901
[600]	training's rmse: 0.000461147	training's RMSPE: 0.21333	valid_1's rmse: 0.000467013	valid_1's RMSPE: 0.21654
[700]	training's rmse: 0.000455839	training's RMSPE: 0.21087	valid_1's rmse: 0.000462583	valid_1's RMSPE: 0.21449
[800]	training's rmse: 0.000451377	training's RMSPE: 0.20881	valid_1's rmse: 0.000458998	

In [17]:
scores

0.2026

# Model保存

In [25]:
import pickle

for i, model in enumerate(models):
    pickle.dump(model, open(model_path+"/lgbm"+str(i)+".pkl", 'wb'))

# 特徴量保存

In [26]:
pickle.dump(df_train, open(os.path.join(feature_path, "train.pkl"), 'wb'))
pickle.dump(df_test, open(os.path.join(feature_path, "test.pkl"), 'wb'))

In [31]:
X_valid.columns.to_series().to_csv(os.path.join(feature_path, "columns.csv"), index=False)